In [1]:
import os
os.getcwd()

'/lambda_stor/homes/ac.tfeng/git/DrugCell'

In [2]:
# import anndata
import numpy as np
import pandas as pd
from typing import List, Union

from torch.utils.data import Dataset, DataLoader, random_split
from torch.distributions import MultivariateNormal

import os
# os.environ["CUDA_VISIBLE_DEVICES"]="4"

import copy

import tqdm

from sklearn.metrics import roc_auc_score


import matplotlib.pyplot as plt


/homes/ac.tfeng/miniconda3/envs/general/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from codes.utils.util import *
from codes.drugcell_NN import *

if torch.cuda.is_available():
  DEVICE = 'cuda'
else:
  DEVICE = 'cpu'
  
torch.cuda.device_count()

8

In [4]:
!nvidia-smi

Sun Feb 11 12:53:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:1A:00.0 Off |                    0 |
| N/A   52C    P0             236W / 300W |  18195MiB / 32768MiB |     97%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
DEVICE = 'cuda:6'

# Data Loading

## GDSC

In [8]:
def set_col_names_in_multilevel_dataframe(
    df: pd.DataFrame,
    level_map: dict,
    gene_system_identifier: Union[str, List[str]]="Gene_Symbol") -> pd.DataFrame:
    """ Util function that supports loading of the omic data files.
    Returns the input dataframe with the multi-level column names renamed as
    specified by the gene_system_identifier arg.

    Args:
        df (pd.DataFrame): omics dataframe
        level_map (dict): encodes the column level and the corresponding identifier systems
        gene_system_identifier (str or list of str): gene identifier system to use
            options: "Entrez", "Gene_Symbol", "Ensembl", "all", or any list
                     combination of ["Entrez", "Gene_Symbol", "Ensembl"]

    Returns:
        pd.DataFrame: the input dataframe with the specified multi-level column names
    """
    df = df.copy()

    level_names = list(level_map.keys())
    level_values = list(level_map.values())
    n_levels = len(level_names)
    
    if isinstance(gene_system_identifier, list) and len(gene_system_identifier) == 1:
        gene_system_identifier = gene_system_identifier[0]

    # print(gene_system_identifier)
    # import pdb; pdb.set_trace()
    if isinstance(gene_system_identifier, str):
        if gene_system_identifier == "all":
            df.columns = df.columns.rename(level_names, level=level_values)  # assign multi-level col names
        else:
            df.columns = df.columns.get_level_values(level_map[gene_system_identifier])  # retian specific column level
    else:
        assert len(gene_system_identifier) <= n_levels, f"'gene_system_identifier' can't contain more than {n_levels} items."
        set_diff = list(set(gene_system_identifier).difference(set(level_names)))
        assert len(set_diff) == 0, f"Passed unknown gene identifiers: {set_diff}"
        kk = {i: level_map[i] for i in level_map if i in gene_system_identifier}
        # print(list(kk.keys()))
        # print(list(kk.values()))
        df.columns = df.columns.rename(list(kk.keys()), level=kk.values())  # assign multi-level col names
        drop_levels = list(set(level_map.values()).difference(set(kk.values())))
        df = df.droplevel(level=drop_levels, axis=1)
    return df


def load_gene_expression_data(gene_expression_file_path, 
    gene_system_identifier: Union[str, List[str]]="Gene_Symbol",
    sep: str="\t",
    verbose: bool=True) -> pd.DataFrame:
    """
    Returns gene expression data.

    Args:
        gene_system_identifier (str or list of str): gene identifier system to use
            options: "Entrez", "Gene_Symbol", "Ensembl", "all", or any list
                     combination of ["Entrez", "Gene_Symbol", "Ensembl"]

    Returns:
        pd.DataFrame: dataframe with the omic data
    """
    # level_map encodes the relationship btw the column and gene identifier system
    level_map = {"Ensembl": 0, "Entrez": 1, "Gene_Symbol": 2}
    header = [i for i in range(len(level_map))]

    df = pd.read_csv(gene_expression_file_path, sep=sep, index_col=0, header=header)

    df.index.name = "improve_sample_id"  # assign index name
    df = set_col_names_in_multilevel_dataframe(df, level_map, gene_system_identifier)
    if verbose:
        print(f"Gene expression data: {df.shape}")
    return df

In [9]:
gene_express = load_gene_expression_data("data/GDSC/cancer_gene_expression.tsv")
response = pd.read_csv("data/GDSC/response.tsv", sep = '\t')

gdsc_info = pd.read_csv("data/GDSC/GDSC_info.csv")

Gene expression data: (1007, 30805)


/tmp/ipykernel_1252376/2783280313.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  response = pd.read_csv("data/GDSC/response.tsv", sep = '\t')


In [10]:
drug_ecfp4_nbits512 = pd.read_csv("data/GDSC/drug_ecfp4_nbits512.tsv", sep = '\t', index_col=0)
drug_tensor = torch.tensor(drug_ecfp4_nbits512.to_numpy())

In [11]:
gdsc = pd.merge(gene_express, gdsc_info, how="inner", left_on=["improve_sample_id"], right_on=['ModelID'])
gdsc_x = gdsc.loc[:, gene_express.columns]
gdsc_y = gdsc.loc[:, 'DepmapModelType']

## Gene ontology

In [12]:
training_file = "data/drugcell_train.txt"
testing_file = "data/drugcell_test.txt"
val_file = "data/drugcell_val.txt"
cell2id_file = "data/cell2ind.txt"
drug2id_file = "data/drug2ind.txt"
genotype_file = "data/cell2mutation.txt"
fingerprint_file = "data/drug2fingerprint.txt"
onto_file = "data/drugcell_ont.txt"
gene2id_file = "data/gene2ind.txt"

train_data, feature_dict, cell2id_mapping, drug2id_mapping = prepare_train_data(training_file, 
                                                                  testing_file, cell2id_file, 
                                                                  drug2id_file)

gene2id_mapping = load_mapping(gene2id_file)

# load cell/drug features
cell_features = np.genfromtxt(genotype_file, delimiter=',')
drug_features = np.genfromtxt(fingerprint_file, delimiter=',')

num_cells = len(cell2id_mapping)
num_drugs = len(drug2id_mapping)
num_genes = len(gene2id_mapping)
drug_dim = len(drug_features[0,:])

# load ontology
dG, root, term_size_map, \
    term_direct_gene_map = load_ontology(onto_file, 
                                         gene2id_mapping)

Total number of cell lines = 1225
Total number of drugs = 684
There are 3008 genes
There are 1 roots: GO:0008150
There are 2086 terms
There are 1 connected componenets


### Align gene id with GDCS

In [13]:
cancer_2_idx = {}
idx_2_cancer = {}
cancer_type_idx = []

i = 0
for cancer in gdsc_y:
    if cancer not in cancer_2_idx:
        cancer_2_idx[cancer] = i
        idx_2_cancer[i] = cancer
        cancer_type_idx.append(i)
        
        i += 1
    else:
        cancer_type_idx.append(cancer_2_idx[cancer])

In [14]:
gene_intersect_list = list(set(gene2id_mapping.keys()) & set(gene_express.columns))
gdsc_tensor = torch.zeros(gene_express.shape[0], num_genes)
for gene in gene_intersect_list:
    idx = gene2id_mapping[gene]
    gdsc_tensor[:,idx] = torch.tensor(gene_express[gene])

In [15]:
gdsc_row_key_id = {k: v for v, k in enumerate(gene_express.index)}
gdsc_row_id_key = {v: k for v, k in enumerate(gene_express.index)}

chem_row_key_id = {k: v for v, k in enumerate(drug_ecfp4_nbits512.index)}
chem_row_id_key = {v: k for v, k in enumerate(drug_ecfp4_nbits512.index)}

In [16]:
response_gdcs2 = response[response['source'] == 'GDSCv2'].loc[:,['improve_sample_id', 
                                                             'improve_chem_id',
                                                             'auc']]
response_gdcs2 = response_gdcs2.replace({'improve_sample_id': gdsc_row_key_id})
response_gdcs2 = response_gdcs2.replace({'improve_chem_id': chem_row_key_id})
response_gdcs2 = torch.tensor(response_gdcs2.to_numpy())

class GDSCData(Dataset):
    
    def __init__(self, response, gene_tensor, chem_tensor):
        self.response = response
        self.gene_tensor = gene_tensor
        self.chem_tensor = chem_tensor
        
    def __len__(self):
        return self.response.shape[0]
    
    def __getitem__(self, index):
        sample = self.response[index,:]
        
        X_gene = self.gene_tensor[sample[0].long() ,:]
        X_chem = self.chem_tensor[sample[1].long() ,:]
        
        y = sample[2]

        X = torch.cat((X_gene, X_chem), 0)
        
        return X, y

In [17]:
train_gdcs_idx = torch.unique(response_gdcs2[:,0], sorted=False)[:423]
test_gdcs_idx = torch.unique(response_gdcs2[:,0], sorted=False)[423:]

In [18]:
gdsc_data = GDSCData(response_gdcs2, gdsc_tensor, drug_tensor)
gdsc_data_train = GDSCData(response_gdcs2[torch.isin(response_gdcs2[:,0], train_gdcs_idx)].float(), gdsc_tensor, drug_tensor)
gdsc_data_test = GDSCData(response_gdcs2[torch.isin(response_gdcs2[:,0], test_gdcs_idx)].float(), gdsc_tensor, drug_tensor)

In [19]:
train_loader = DataLoader(gdsc_data_train, batch_size=8192, shuffle=True)
test_loader = DataLoader(gdsc_data_test, batch_size=8192, shuffle=False)
(inputdata, response) = next(iter(test_loader))

In [20]:
np.savetxt("hpo_data/response_gdcs2.csv", response_gdcs2.numpy(), delimiter=",")
np.savetxt("hpo_data/gdsc_tensor.csv", gdsc_tensor.numpy(), delimiter=",")
np.savetxt("hpo_data/drug_tensor.csv", drug_tensor.numpy(), delimiter=",")


In [21]:
response_gdcs2

tensor([[9.4900e+02, 6.7400e+02, 5.8330e-01],
        [9.4900e+02, 6.7400e+02, 6.0260e-01],
        [9.4900e+02, 6.7400e+02, 4.3030e-01],
        ...,
        [5.1500e+02, 2.1400e+02, 9.1050e-01],
        [5.1500e+02, 1.4430e+03, 9.5660e-01],
        [5.1500e+02, 8.5500e+02, 8.4260e-01]], dtype=torch.float64)

In [22]:
from numpy import float32


arr = np.loadtxt("hpo_data/response_gdcs2.csv",
                 delimiter=",", dtype=float32)

In [23]:
arr

array([[9.490e+02, 6.740e+02, 5.833e-01],
       [9.490e+02, 6.740e+02, 6.026e-01],
       [9.490e+02, 6.740e+02, 4.303e-01],
       ...,
       [5.150e+02, 2.140e+02, 9.105e-01],
       [5.150e+02, 1.443e+03, 9.566e-01],
       [5.150e+02, 8.550e+02, 8.426e-01]], dtype=float32)

# Model

In [24]:
class Drugcell_Vae(nn.Module):

    def __init__(self, term_size_map, term_direct_gene_map, dG, ngene, ndrug, root, 
                 num_hiddens_genotype, num_hiddens_drug, num_hiddens_final, 
                 n_class, inter_loss_penalty = 0.2):

        super().__init__()

        self.root = root
        self.num_hiddens_genotype = num_hiddens_genotype
        self.num_hiddens_drug = num_hiddens_drug
        
        
        self.num_hiddens_final = num_hiddens_final
        self.n_class = n_class
        self.inter_loss_penalty = inter_loss_penalty
        self.dG = copy.deepcopy(dG)

        # dictionary from terms to genes directly annotated with the term
        self.term_direct_gene_map = term_direct_gene_map

        self.term_visit_count = {}
        self.init_term_visits(term_size_map)
        
        # calculate the number of values in a state (term): term_size_map is the number of all genes annotated with the term
        self.term_dim_map = {}
        self.cal_term_dim(term_size_map)

        # ngenes, gene_dim are the number of all genes
        self.gene_dim = ngene
        self.drug_dim = ndrug

        # add modules for neural networks to process genotypes
        self.contruct_direct_gene_layer()
        self.construct_NN_graph(self.dG)

        # add modules for neural networks to process drugs
        self.construct_NN_drug()

        # add modules for final layer TODO: modify it into VAE
        final_input_size = num_hiddens_genotype + num_hiddens_drug[-1]
        self.add_module('final_linear_layer', nn.Linear(final_input_size, num_hiddens_final * 2))
        self.add_module('final_batchnorm_layer', nn.BatchNorm1d(num_hiddens_final * 2))
        self.add_module('final_aux_linear_layer', nn.Linear(num_hiddens_final * 2, 1))
        self.add_module('final_linear_layer_output', nn.Linear(1, 1))
        
        self.decoder_affine = nn.Linear(num_hiddens_final, 1)

    def init_term_visits(self, term_size_map):
        
        for term in term_size_map:
            self.term_visit_count[term] = 0
    
    # calculate the number of values in a state (term)
    def cal_term_dim(self, term_size_map):

        for term, term_size in term_size_map.items():
            num_output = self.num_hiddens_genotype

            # log the number of hidden variables per each term
            num_output = int(num_output)
#            print("term\t%s\tterm_size\t%d\tnum_hiddens\t%d" % (term, term_size, num_output))
            self.term_dim_map[term] = num_output


    # build a layer for forwarding gene that are directly annotated with the term
    def contruct_direct_gene_layer(self):

        for term, gene_set in self.term_direct_gene_map.items():
            if len(gene_set) == 0:
                print('There are no directed asscoiated genes for', term)
                sys.exit(1)

            # if there are some genes directly annotated with the term, add a layer taking in all genes and forwarding out only those genes
            self.add_module(term+'_direct_gene_layer', nn.Linear(self.gene_dim, len(gene_set)))


    # add modules for fully connected neural networks for drug processing
    def construct_NN_drug(self):
        input_size = self.drug_dim

        for i in range(len(self.num_hiddens_drug)):
            self.add_module('drug_linear_layer_' + str(i+1), nn.Linear(input_size, self.num_hiddens_drug[i]))
            self.add_module('drug_batchnorm_layer_' + str(i+1), nn.BatchNorm1d(self.num_hiddens_drug[i]))
            self.add_module('drug_aux_linear_layer1_' + str(i+1), nn.Linear(self.num_hiddens_drug[i],1))
            self.add_module('drug_aux_linear_layer2_' + str(i+1), nn.Linear(1,1))

            input_size = self.num_hiddens_drug[i]

    # start from bottom (leaves), and start building a neural network using the given ontology
    # adding modules --- the modules are not connected yet
    def construct_NN_graph(self, dG):

        self.term_layer_list = []   # term_layer_list stores the built neural network
        self.term_neighbor_map = {}

        # term_neighbor_map records all children of each term
        for term in dG.nodes():
            self.term_neighbor_map[term] = []
            for child in dG.neighbors(term):
                self.term_neighbor_map[term].append(child)

        while True:
            leaves = [n for n in dG.nodes() if dG.out_degree(n) == 0]
            #leaves = [n for n,d in dG.out_degree().items() if d==0]
            #leaves = [n for n,d in dG.out_degree() if d==0]

            if len(leaves) == 0:
                break

            self.term_layer_list.append(leaves)

            for term in leaves:

                # input size will be #chilren + #genes directly annotated by the term
                input_size = 0

                for child in self.term_neighbor_map[term]:
                    input_size += self.term_dim_map[child]

                if term in self.term_direct_gene_map:
                    input_size += len(self.term_direct_gene_map[term])

                # term_hidden is the number of the hidden variables in each state
                term_hidden = self.term_dim_map[term]

                self.add_module(term+'_linear_layer', nn.Linear(input_size, term_hidden))
                self.add_module(term+'_batchnorm_layer', nn.BatchNorm1d(term_hidden))
                self.add_module(term+'_aux_linear_layer1', nn.Linear(term_hidden, term_hidden))
                self.add_module(term+'_aux_linear_layer2', nn.Linear(term_hidden, 1))

            dG.remove_nodes_from(leaves)


    # definition of encoder
    def encoder(self, x):
        gene_input = x.narrow(1, 0, self.gene_dim)
        drug_input = x.narrow(1, self.gene_dim, self.drug_dim)
        
        # define forward function for genotype dcell #############################################
        term_gene_out_map = {}

        for term, _ in self.term_direct_gene_map.items():
            term_gene_out_map[term] = self._modules[term + '_direct_gene_layer'](gene_input)

        term_NN_out_map = {}
        aux_out_map = {}

        for i, layer in enumerate(self.term_layer_list):

            for term in layer:

                child_input_list = []

                self.term_visit_count[term] += 1
                
                for child in self.term_neighbor_map[term]:
                    child_input_list.append(term_NN_out_map[child])

                if term in self.term_direct_gene_map:
                    child_input_list.append(term_gene_out_map[term])

                child_input = torch.cat(child_input_list,1)

                term_NN_out = self._modules[term+'_linear_layer'](child_input)

                Tanh_out = torch.tanh(term_NN_out)
                term_NN_out_map[term] = self._modules[term+'_batchnorm_layer'](Tanh_out)
                aux_layer1_out = torch.tanh(self._modules[term+'_aux_linear_layer1'](term_NN_out_map[term]))
                aux_out_map[term] = self._modules[term+'_aux_linear_layer2'](aux_layer1_out)

        drug_out = drug_input

        for i in range(1, len(self.num_hiddens_drug)+1, 1):
            drug_out = self._modules['drug_batchnorm_layer_'+str(i)]( torch.tanh(self._modules['drug_linear_layer_' + str(i)](drug_out)))
            term_NN_out_map['drug_'+str(i)] = drug_out

            aux_layer1_out = torch.tanh(self._modules['drug_aux_linear_layer1_'+str(i)](drug_out))
            aux_out_map['drug_'+str(i)] = self._modules['drug_aux_linear_layer2_'+str(i)](aux_layer1_out)


        # connect two neural networks at the top #################################################
        final_input = torch.cat((term_NN_out_map[self.root], drug_out), 1)

        out = self._modules['final_batchnorm_layer'](torch.tanh(self._modules['final_linear_layer'](final_input)))
        term_NN_out_map['final'] = out

        aux_layer_out = torch.tanh(self._modules['final_aux_linear_layer'](out))
        aux_out_map['final'] = self._modules['final_linear_layer_output'](aux_layer_out)

        return aux_out_map, term_NN_out_map
    
    def forward(self, x):
        
        aux_out_map, term_NN_out_map = self.encoder(x)
        
        mu = term_NN_out_map['final'][..., :self.num_hiddens_final]
        log_var = term_NN_out_map['final'][..., :self.num_hiddens_final]  # T X batch X z_dim
        std_dec = log_var.mul(0.5).exp_()
        # std_dec = 1
        
        latent = MultivariateNormal(loc = mu, 
                                    scale_tril=torch.diag_embed(std_dec))
        z = latent.rsample()
        
        recon_mean = self.decoder_affine(z)
        recon_mean = F.sigmoid(recon_mean)

        return recon_mean, mu, log_var, aux_out_map, term_NN_out_map
    
    def loss_log_vae(self, recon_mean, y, mu, log_var, beta = 0.001):
        # y: true labels
        ori_y_shape = y.shape
        
        class_loss = F.mse_loss(recon_mean.view(-1), 
                                     y.reshape(-1), reduction = 'none').div(np.log(2)).view(*ori_y_shape)
        
        KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp(), 
                              dim = -1)
        
        log_loss = class_loss + beta * KLD
        log_loss = torch.mean(torch.logsumexp(log_loss, 0))
        
        return log_loss
    
    def intermediate_loss(self, aux_out_map, y):
        
        inter_loss = 0
        for name, output in aux_out_map.items():
            if name == 'final':
                inter_loss += 0
            else: # change 0.2 to smaller one for big terms
                ori_y_shape = y.shape
        
                term_loss = F.mse_loss(output.view(-1), 
                                             y.reshape(-1), 
                                             reduction = 'none').div(np.log(2)).view(*ori_y_shape)
                inter_loss += term_loss

        return inter_loss

# Training

In [25]:
def create_term_mask(term_direct_gene_map, gene_dim, device):

    term_mask_map = {}

    for term, gene_set in term_direct_gene_map.items():

        mask = torch.zeros(len(gene_set), gene_dim)

        for i, gene_id in enumerate(gene_set):
            mask[i, gene_id] = 1

        mask_gpu = torch.autograd.Variable(mask)

        term_mask_map[term] = mask_gpu.to(device)

    return term_mask_map

In [26]:
drug_ecfp4_nbits512.shape[1]

512

In [96]:
torch.manual_seed(1)

num_hiddens_genotype = 6
num_hiddens_final = 6
drug_hiddens='100,50,6'
num_hiddens_drug = list(map(int, drug_hiddens.split(',')))
# num_hiddens_drug = 6
num_drugs = drug_ecfp4_nbits512.shape[1]

model = Drugcell_Vae(term_size_map, term_direct_gene_map, dG, num_genes, num_drugs, 
                 root, num_hiddens_genotype, num_hiddens_drug, num_hiddens_final, 
                 n_class = len(cancer_2_idx))
model.to(DEVICE)
term_mask_map = create_term_mask(model.term_direct_gene_map, num_genes, device = DEVICE)


learning_rate = 0.001
torch.manual_seed(0)
training_loss_list = []
testing_loss_list = []
epoch_list = []
accu_list = []
train_epochs = 50
best_loss = 1000


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99), eps=1e-05)
term_mask_map = create_term_mask(model.term_direct_gene_map, gene_dim=num_genes, device=DEVICE)

optimizer.zero_grad()

for name, param in model.named_parameters():
    term_name = name.split('_')[0]

    if '_direct_gene_layer.weight' in name:
        param.data = torch.mul(param.data, term_mask_map[term_name].to(DEVICE)) * 0.1
    else:
        param.data = param.data * 0.1

mse_tmp_testing = torch.tensor(0, device=DEVICE)
# tepoch = tqdm.tqdm(range(train_epochs))
for epoch in range(train_epochs):
    # Train
    model.train()
    train_predict = torch.zeros(0, 0).to(DEVICE)

    tloader = tqdm.tqdm(enumerate(train_loader))
    for i, (data, response) in tloader:
        # Convert torch tensor to Variable

        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer

        # Here term_NN_out_map is a dictionary
        recon_mean, mu, log_var, aux_out_map, term_NN_out_map = model(data.to(DEVICE))

        if train_predict.size()[0] == 0:
            train_predict = aux_out_map["final"].data
        else:
            train_predict = torch.cat([train_predict, aux_out_map["final"].data], dim=0)

        total_loss = 0

        loss_vae = model.loss_log_vae(
            recon_mean=recon_mean, y=response.to(DEVICE), mu=mu, log_var=log_var, beta=0.001
        )

        loss_intermidiate = model.intermediate_loss(aux_out_map, response.to(DEVICE))

        total_loss = torch.mean(loss_vae + model.inter_loss_penalty * loss_intermidiate)
        
        tmp_loss = total_loss.item()
        # tepoch.set_postfix({"Epoch": epoch, 
        #                     "Training Loss": tmp_loss, 
        #                     "Testing Loss": mse_tmp_testing.item()})
        
        total_loss.backward()

        for name, param in model.named_parameters():
            if "_direct_gene_layer.weight" not in name:
                continue
            term_name = name.split("_")[0]
            # print name, param.grad.data.size(), term_mask_map[term_name].size()
            param.grad.data = torch.mul(param.grad.data, term_mask_map[term_name])

        optimizer.step()

        if i % 6 == 0:
            with torch.no_grad():
                (inputdata, response) = next(iter(test_loader))
                recon_mean, mu, log_var, aux_out_map, term_NN_out_map = model(inputdata.to(DEVICE))

                mse_tmp_testing = F.mse_loss(recon_mean.detach().squeeze().cpu(), response.squeeze())

                tloader.set_postfix({"Epoch": epoch, 
                                    "Training Loss": tmp_loss, 
                                    "Testing Loss": mse_tmp_testing.item()})
                
                training_loss_list.append(tmp_loss)
                testing_loss_list.append(mse_tmp_testing.item())
                epoch_list.append(epoch)
    with torch.no_grad():
        (inputdata, response) = next(iter(test_loader))
        recon_mean, mu, log_var, aux_out_map, term_NN_out_map = model(inputdata.to(DEVICE))

        mse_tmp_testing = F.mse_loss(recon_mean.detach().squeeze().cpu(), response.squeeze())
        
        if mse_tmp_testing < best_loss:
            torch.save(model, "gdsc_drug_epoch_new.pt")
    # if epoch % 10 == 0:
    # torch.save(model, "gdsc_50.pt")

187it [10:12,  3.27s/it, Epoch=0, Training Loss=17.8, Testing Loss=0.0617]
187it [10:08,  3.25s/it, Epoch=1, Training Loss=23.1, Testing Loss=0.0301]
187it [10:08,  3.26s/it, Epoch=2, Training Loss=18.1, Testing Loss=0.022] 
187it [10:32,  3.38s/it, Epoch=3, Training Loss=18, Testing Loss=0.0191]  
187it [10:41,  3.43s/it, Epoch=4, Training Loss=18.7, Testing Loss=0.0171]
187it [10:12,  3.27s/it, Epoch=5, Training Loss=18.9, Testing Loss=0.0159]
187it [10:09,  3.26s/it, Epoch=6, Training Loss=16.5, Testing Loss=0.0149]
187it [10:10,  3.26s/it, Epoch=7, Training Loss=23.8, Testing Loss=0.0153]
187it [10:08,  3.25s/it, Epoch=8, Training Loss=18.2, Testing Loss=0.0144]
187it [10:09,  3.26s/it, Epoch=9, Training Loss=18, Testing Loss=0.015]   
187it [10:07,  3.25s/it, Epoch=10, Training Loss=22.2, Testing Loss=0.0142]
187it [10:08,  3.25s/it, Epoch=11, Training Loss=20.1, Testing Loss=0.014] 
187it [10:10,  3.26s/it, Epoch=12, Training Loss=17.3, Testing Loss=0.0147]
187it [10:14,  3.28s/i

In [72]:
model= torch.load("gdsc_drug_epoch_new.pt", map_location=DEVICE)

In [74]:
with torch.no_grad():
    (inputdata, response) = next(iter(test_loader))
    recon_mean, mu, log_var, aux_out_map, term_NN_out_map = model(inputdata.to(DEVICE))

    mse_tmp_testing = F.mse_loss(recon_mean.detach().squeeze().cpu(), response.squeeze())


/homes/ac.tfeng/miniconda3/envs/general/lib/python3.9/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
